In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from nltk.tokenize import word_tokenize
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import Dense, LSTM, SimpleRNN, Input, Embedding, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from nltk.corpus import stopwords
import optuna
import spacy

c:\Users\MSI\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
text_cleaned = pd.read_parquet('data_cleaned.parquet')

In [4]:
text_cleaned

,review_text,review_rate
0,"[mayor, virtud, película, existencia.el, hecho...",negative
1,"[experto, cinéfilo, ,, poco, vez, tanto, juego...",negative
2,"[si, incondicional, humor, estilo, tele, 5.si,...",negative
3,"[saber, pasar, ,, si, gente, dejar, llevar, mo...",negative
4,"[`, `, amanecer, ,, quedo, solo, ,, sentir, fo...",negative
...,...,...
56362,"[pensar, película, hacer, buen, trabajo, derec...",positive
56363,"[malo, parcela, ,, mal, diálogo, ,, malo, actu...",negative
56364,"[católico, enseñado, escuela, primario, parroq...",negative
56365,"[ir, tener, desacuerdo, comentario, anterior, ...",negative


In [5]:
text_vectorizer = TextVectorization(output_mode='int')

In [6]:
text_cleaned['review_text'] = text_cleaned['review_text'].map(lambda corpus: ' '.join(corpus))

In [7]:
text_vectorizer.adapt([text_cleaned['review_text']])

In [8]:
text_cleaned

,review_text,review_rate
0,mayor virtud película existencia.el hecho pode...,negative
1,"experto cinéfilo , poco vez tanto juego sala c...",negative
2,si incondicional humor estilo tele 5.si termin...,negative
3,"saber pasar , si gente dejar llevar moda , si ...",negative
4,"` ` amanecer , quedo solo , sentir fondoun mar...",negative
...,...,...
56362,pensar película hacer buen trabajo derecha.no ...,positive
56363,"malo parcela , mal diálogo , malo actuación , ...",negative
56364,católico enseñado escuela primario parroquial ...,negative
56365,ir tener desacuerdo comentario anterior lado m...,negative


In [9]:
model = Sequential(name='Text_Vectorizing')
model.add(Input(shape=(1,), dtype=tf.string))
model.add(text_vectorizer)

text_vectorized = model.predict([text_cleaned['review_text']])


1762/1762 [==============================] - 9s 5ms/step


In [10]:
size_voc = len(text_vectorizer.get_vocabulary())

In [11]:
y_prediction_rate = OneHotEncoder().fit_transform(np.array(text_cleaned['review_rate']).reshape(-1,1)).toarray()

In [12]:
x_train, x_test, y_train,y_test = train_test_split(text_vectorized.to_tensor().numpy(),y_prediction_rate,random_state=43, train_size=.8)

In [14]:

text_vectorizer = TextVectorization(max_tokens=size_voc, output_mode='int')

# Fit the TextVectorization layer on your text data
text_vectorizer.adapt(text_cleaned['review_text'].values)

# Create a model
model = Sequential(name='Text_Vectorizing')
model.add(Input(shape=(1,), dtype=tf.string))
model.add(text_vectorizer)

# Vectorize the text data
text_vectorized = model.predict([text_cleaned['review_text']])

# Split the data
x_train, x_test, y_train,y_test = train_test_split(text_vectorized.to_tensor().numpy(),y_prediction_rate,random_state=43, train_size=.8)


1762/1762 [==============================] - 16s 9ms/step


In [ ]:
words =dict(enumerate(text_vectorizer.get_vocabulary()))

In [37]:

def objective(trial):
    len_max = trial.suggest_int('MX_LEN', 1, 300, log=True)
    out_dim = trial.suggest_int('OUT_DIM', 1, 300, log=True)

    x_train_padd = sequence.pad_sequences(x_train, maxlen=len_max, padding='post', truncating='post')
    x_test_padd = sequence.pad_sequences(x_test, maxlen=len_max, padding='post', truncating='post')

    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)

    al_optimizer = trial.suggest_categorical('optimizer', ['adam', 'sgd', 'rmsprop', 'adafactor'])

    model_text = tf.keras.Sequential()
    model_text.add(Embedding(input_dim=size_voc + 1, output_dim=out_dim, input_length=len_max))

    num_hidden = trial.suggest_int('num_ner',1, 200, log=True)

    model_text.add(LSTM(num_hidden))  # You can adjust the number of units as needed

    model_text.add(Dense(2, activation='softmax'))

    # Compile and train the model
    if al_optimizer == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif al_optimizer == 'sgd':
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    elif al_optimizer == 'rmsprop':
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    elif al_optimizer == 'adafactor':
        optimizer = tf.keras.optimizers.Adafactor(learning_rate=learning_rate)

    model_text.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    model_text.fit(x_train_padd, y_train, epochs=3, batch_size=120, verbose=0)

    # Evaluate the model on the test set
    _, accuracy = model_text.evaluate(x_test_padd, y_test)

    return accuracy



In [38]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Get the best parameters
best_params = study.best_params
print(f"Best hyperparameters: {best_params}")

# Get the best accuracy
best_accuracy = study.best_value
print(f"Best accuracy: {best_accuracy}")

[I 2024-02-18 22:18:37,562] A new study created in memory with name: no-name-9f422a55-9dc5-40b4-a9a2-9e5aeb78d29f


353/353 [==============================] - 1s 3ms/step - loss: 0.5607 - accuracy: 0.7046


[I 2024-02-18 22:20:25,284] Trial 0 finished with value: 0.7046301364898682 and parameters: {'MX_LEN': 12, 'OUT_DIM': 150, 'learning_rate': 0.00023639048369169217, 'optimizer': 'rmsprop', 'num_ner': 83}. Best is trial 0 with value: 0.7046301364898682.


353/353 [==============================] - 2s 4ms/step - loss: 0.5281 - accuracy: 0.7334


[I 2024-02-18 22:20:59,626] Trial 1 finished with value: 0.7333688139915466 and parameters: {'MX_LEN': 52, 'OUT_DIM': 7, 'learning_rate': 5.379937934576627e-05, 'optimizer': 'adam', 'num_ner': 27}. Best is trial 1 with value: 0.7333688139915466.


...

353/353 [==============================] - 4s 11ms/step - loss: 0.6928 - accuracy: 0.5161


[I 2024-02-19 01:25:16,769] Trial 98 finished with value: 0.5161433219909668 and parameters: {'MX_LEN': 199, 'OUT_DIM': 24, 'learning_rate': 0.00036358766358871036, 'optimizer': 'sgd', 'num_ner': 7}. Best is trial 72 with value: 0.8592336177825928.


353/353 [==============================] - 3s 7ms/step - loss: 0.4202 - accuracy: 0.8293


[I 2024-02-19 01:26:51,640] Trial 99 finished with value: 0.8293418288230896 and parameters: {'MX_LEN': 115, 'OUT_DIM': 30, 'learning_rate': 9.650738281213013e-05, 'optimizer': 'adam', 'num_ner': 12}. Best is trial 72 with value: 0.8592336177825928.


Best hyperparameters: {'MX_LEN': 142, 'OUT_DIM': 3, 'learning_rate': 0.0007116182026416922, 'optimizer': 'adam', 'num_ner': 9}
Best accuracy: 0.8592336177825928


In [21]:
MX_LEN= 142

x_train_padd = sequence.pad_sequences(x_train, maxlen=MX_LEN, padding='post',
                                         truncating = 'post')

x_test_padd = sequence.pad_sequences(x_test, maxlen=MX_LEN, padding='post',
                                     truncating='post')


In [61]:
model_text = Sequential()

model_text.add(Embedding(input_dim=size_voc+1,output_dim=3,input_length=103))
model_text.add(LSTM(9))
model_text.add(Dense(2, activation='softmax'))

optimiz = tf.optimizers.Adam(learning_rate=0.00071)

model_text.compile(loss='categorical_crossentropy', optimizer=optimiz, metrics=['accuracy'])

In [62]:
model_text.summary()

Model: "sequential_206"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_206 (Embedding)   (None, 103, 3)            866997    
                                                                 
 lstm_101 (LSTM)             (None, 9)                 468       
                                                                 
 dense_206 (Dense)           (None, 2)                 20        
                                                                 
Total params: 867485 (3.31 MB)
Trainable params: 867485 (3.31 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [63]:
history = model_text.fit(x_train_padd, y_train, batch_size=120, epochs=3, validation_data=(x_test_padd, y_test))

Epoch 1/3
376/376 [==============================] - 17s 39ms/step - loss: 0.6152 - accuracy: 0.6192 - val_loss: 0.4189 - val_accuracy: 0.8128
Epoch 2/3
376/376 [==============================] - 14s 37ms/step - loss: 0.3107 - accuracy: 0.8743 - val_loss: 0.3457 - val_accuracy: 0.8465
Epoch 3/3
376/376 [==============================] - 15s 39ms/step - loss: 0.1659 - accuracy: 0.9424 - val_loss: 0.3802 - val_accuracy: 0.8370


In [64]:
lemmer = spacy.load('es_core_news_sm')

def lematizer_text(rows):
    
    doc = [word.lemma_ for word in lemmer(rows)]

    return doc

In [65]:
stopwords_spanish = stopwords.words('spanish')
def clean_stopWords(rows):
    no_stops = []
    for word in rows:
        if word not in stopwords_spanish:
            no_stops.append(word)
    return no_stops

In [72]:

probe_padd = sequence.pad_sequences(model.predict([' '.join(clean_stopWords(lematizer_text('Habría perdido mi tiempo porque no veo películas, pero cuando ví esta película pensé que quizás ver películas puede ser hasta divertidas y sí, valió la pena mi tiempo')))]), maxlen=103, padding='post', truncating='post')

rest = model_text.predict(probe_padd.reshape(1,103,1))

if rest[0][0]> rest[0][1]:
    print(rest[0][0],'Negativo')
else:
    print(rest[0][1],'Positivo')

1/1 [==============================] - 0s 26ms/step


0.5389313 Positivo


In [74]:
model_text.save('RNN_model_prueba.h5')